In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF, RationalQuadratic, WhiteKernel
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import ipywidgets as widgets
from ipywidgets import interact, interactive_output, Layout, HBox, VBox, Label, Button
from IPython.display import display, clear_output
import warnings


In [ ]:
warnings.filterwarnings('ignore')
# Set up the font and style for plots
#plt.style.use('seaborn-whitegrid')

def generate_fitness_landscape(dimensions=2, epistasis=1.0, ruggedness=1.0, sparsity=0.0):
    """
    Generates a synthetic fitness landscape with controllable epistasis, ruggedness, and sparsity.
    """
    def fitness_function(X):
        # Base fitness landscape
        fitness = np.zeros((X.shape[0], 1))
        for i in range(dimensions):
            xi = X[:, i]
            base = np.sin(5 * xi)
            epi = epistasis * np.sin(5 * np.prod(X, axis=1))
            rug = ruggedness * np.sin(10 * xi)
            fitness[:, 0] += base + epi + rug
        # Apply sparsity by zeroing out small values
        fitness[np.abs(fitness) < sparsity] = 0
        # Exponential decay to simulate realistic landscapes
        fitness *= np.exp(-np.sum(X**2, axis=1)).reshape(-1, 1)
        return fitness.reshape(-1, 1)
    return fitness_function

def initialize_samples(initial_samples, dimensions, sampling_method):
    """
    Initializes sample points using the specified sampling method.
    """
    if sampling_method == 'Random':
        X_sample = np.random.uniform(-2, 2, (initial_samples, dimensions))
    elif sampling_method == 'Latin Hypercube':
        from pyDOE import lhs
        X_sample = lhs(dimensions, samples=initial_samples, criterion='maximin')
        X_sample = X_sample * 4 - 2  # Scale to [-2, 2]
    elif sampling_method == 'Sobol':
        from scipy.stats import qmc
        sampler = qmc.Sobol(d=dimensions, scramble=False)
        X_sample = sampler.random_base2(m=int(np.log2(initial_samples)))
        X_sample = X_sample * 4 - 2  # Scale to [-2, 2]
    else:
        X_sample = np.random.uniform(-2, 2, (initial_samples, dimensions))
    return X_sample

def select_surrogate_model(model_name, kernel_choice='Matern', length_scale=1.0, noise_level=1e-5):
    """
    Selects and initializes the surrogate model based on user choice.
    """
    if model_name == 'Gaussian Process':
        if kernel_choice == 'Matern':
            kernel = Matern(length_scale=length_scale) + WhiteKernel(noise_level=noise_level)
        elif kernel_choice == 'RBF':
            kernel = RBF(length_scale=length_scale) + WhiteKernel(noise_level=noise_level)
        elif kernel_choice == 'RationalQuadratic':
            kernel = RationalQuadratic(length_scale=length_scale) + WhiteKernel(noise_level=noise_level)
        else:
            kernel = Matern(length_scale=length_scale) + WhiteKernel(noise_level=noise_level)
        model = GaussianProcessRegressor(kernel=kernel)
    elif model_name == 'Random Forest':
        model = RandomForestRegressor(n_estimators=100)
    elif model_name == 'Neural Network':
        model = MLPRegressor(hidden_layer_sizes=(50,50), max_iter=1000, alpha=1e-4)
    else:
        model = GaussianProcessRegressor()
    return model

def compute_acquisition(acquisition_function, mu, sigma, Y_sample, kappa=2.0, xi=0.01):
    """
    Computes the acquisition function values.
    """
    if acquisition_function == 'UCB':
        return mu + kappa * sigma
    elif acquisition_function == 'Expected Improvement':
        mu_sample_opt = np.max(Y_sample)
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
        return ei
    elif acquisition_function == 'Probability of Improvement':
        mu_sample_opt = np.max(Y_sample)
        Z = (mu - mu_sample_opt - xi) / sigma
        pi = norm.cdf(Z)
        return pi
    elif acquisition_function == 'Thompson Sampling':
        samples = np.random.normal(mu, sigma)
        return samples
    elif acquisition_function == 'Random':
        return np.random.rand(*mu.shape)
    else:
        return mu + kappa * sigma  # Default to UCB

def estimate_uncertainty(model, X, model_name):
    """
    Estimates the mean and uncertainty (standard deviation) of the model's predictions.
    """
    if model_name == 'Gaussian Process':
        mu, sigma = model.predict(X, return_std=True)
    elif model_name == 'Random Forest':
        # Get predictions from all trees
        all_preds = np.array([tree.predict(X) for tree in model.estimators_])
        mu = np.mean(all_preds, axis=0)
        sigma = np.std(all_preds, axis=0)
    elif model_name == 'Neural Network':
        # Use Monte Carlo Dropout for uncertainty estimation
        T = 10  # Number of forward passes
        preds = []
        for _ in range(T):
            preds.append(model.predict(X))
        mu = np.mean(preds, axis=0)
        sigma = np.std(preds, axis=0)
    else:
        mu, sigma = model.predict(X, return_std=True)
    return mu.reshape(-1, 1), sigma.reshape(-1, 1)

def bayesian_optimization(fitness_function, X_sample, Y_sample, model, model_name, acquisition_function, iterations, batch_size, kappa, xi):
    """
    Performs Bayesian optimization to select the next sampling points.
    Returns updated samples and acquisition function values along with idx_next.
    """
    # Initialize scaler for Neural Network
    scaler = StandardScaler()
    if model_name == 'Neural Network':
        X_sample_scaled = scaler.fit_transform(X_sample)
    
    progress = widgets.IntProgress(value=0, min=0, max=iterations, description='Progress:', bar_style='info')
    display(progress)
    
    for iteration in range(iterations):
        # Fit the model
        if model_name == 'Neural Network':
            model.fit(X_sample_scaled, Y_sample.ravel())
        else:
            model.fit(X_sample, Y_sample.ravel())
        
        # Predict over the grid
        if model_name == 'Neural Network':
            X_grid_scaled = scaler.transform(X_grid)
            mu, sigma = estimate_uncertainty(model, X_grid_scaled, model_name)
        else:
            mu, sigma = estimate_uncertainty(model, X_grid, model_name)
        
        # Compute acquisition function
        acquisition = compute_acquisition(acquisition_function, mu, sigma, Y_sample, kappa, xi)
        
        # Select next sample points
        idx_next = np.argpartition(-acquisition.ravel(), batch_size-1)[:batch_size]
        X_next = X_grid[idx_next]
        Y_next = fitness_function(X_next)
        
        # Update samples
        X_sample = np.vstack((X_sample, X_next))
        Y_sample = np.vstack((Y_sample, Y_next))
        if model_name == 'Neural Network':
            X_sample_scaled = scaler.fit_transform(X_sample)
        
        progress.value += 1  # Update progress bar
    
    progress.close()
    return X_sample, Y_sample, mu, sigma, acquisition, idx_next

def plot_results(X_sample, Y_sample, mu, sigma, acquisition, dimensions, model_name, acquisition_function, color_map, batch_size, idx_next):
    """
    Plots the results of the optimization.
    """
    if dimensions == 1:
        plt.figure(figsize=(12, 8))
        # Plot the fitness landscape and surrogate model
        plt.subplot(2, 1, 1)
        plt.title('Fitness Landscape and Surrogate Model')
        plt.plot(X_grid, Y_grid_true, 'r--', label='True Fitness Function')
        plt.plot(X_grid, mu, color='#1f77b4', label='Surrogate Prediction')
        plt.fill_between(X_grid.ravel(), (mu - sigma).ravel(), (mu + sigma).ravel(),
                         alpha=0.2, color='#1f77b4', label='Uncertainty')
        plt.scatter(X_sample, Y_sample, c='black', s=50, zorder=10, label='Samples')
        plt.legend()
        
        # Plot the acquisition function
        plt.subplot(2, 1, 2)
        plt.title(f'Acquisition Function: {acquisition_function}')
        plt.plot(X_grid, acquisition, 'g-', label='Acquisition Function')
        # Plot the next sample(s)
        plt.scatter(X_grid[idx_next], acquisition[idx_next], c='red', s=50, zorder=10, label='Next Sample(s)')
        plt.legend()
        plt.tight_layout()
        plt.show()
    elif dimensions == 2:
        fig = plt.figure(figsize=(16, 12))
        # Reshape grid for plotting
        grid = [X_grid[:, i].reshape(len(x), len(x)) for i in range(dimensions)]
        
        # Plot the true fitness landscape
        ax1 = fig.add_subplot(2, 2, 1, projection='3d')
        ax1.plot_surface(*grid, Y_grid_true.reshape(len(x), len(x)), cmap=color_map, alpha=0.7)
        ax1.scatter(X_sample[:, 0], X_sample[:, 1], Y_sample.ravel(), c='red', s=50, label='Samples')
        ax1.set_title('True Fitness Landscape')
        ax1.set_xlabel('X1')
        ax1.set_ylabel('X2')
        ax1.set_zlabel('Fitness')
        ax1.legend()
        
        # Plot the surrogate model prediction
        ax2 = fig.add_subplot(2, 2, 2, projection='3d')
        ax2.plot_surface(*grid, mu.reshape(len(x), len(x)), cmap=color_map, alpha=0.7)
        ax2.scatter(X_sample[:, 0], X_sample[:, 1], Y_sample.ravel(), c='red', s=50, label='Samples')
        ax2.set_title(f'Surrogate Model Prediction ({model_name})')
        ax2.set_xlabel('X1')
        ax2.set_ylabel('X2')
        ax2.set_zlabel('Predicted Fitness')
        ax2.legend()
        
        # Plot the acquisition function
        ax3 = fig.add_subplot(2, 2, 3)
        cont = ax3.contourf(grid[0], grid[1], acquisition.reshape(len(x), len(x)), cmap=color_map)
        ax3.scatter(X_sample[:, 0], X_sample[:, 1], c='red', s=50, label='Samples')
        fig.colorbar(cont, ax=ax3)
        ax3.set_title(f'Acquisition Function: {acquisition_function}')
        ax3.set_xlabel('X1')
        ax3.set_ylabel('X2')
        ax3.legend()
        
        # Plot the standard deviation (uncertainty)
        ax4 = fig.add_subplot(2, 2, 4)
        cont2 = ax4.contourf(grid[0], grid[1], sigma.reshape(len(x), len(x)), cmap=color_map)
        ax4.scatter(X_sample[:, 0], X_sample[:, 1], c='red', s=50, label='Samples')
        fig.colorbar(cont2, ax=ax4)
        ax4.set_title('Predictive Uncertainty')
        ax4.set_xlabel('X1')
        ax4.set_ylabel('X2')
        ax4.legend()
        
        plt.tight_layout()
        plt.show()
    else:
        print("Visualization not supported for dimensions greater than 2.")

def interactive_bayesian_optimization(acquisition_function, surrogate_model, kernel_choice,
                                      kappa, xi, epistasis, ruggedness, sparsity, iterations,
                                      batch_size, dimensions, initial_samples, sampling_method,
                                      length_scale, noise_level, random_state, color_scheme):
    """
    Main function to run Bayesian optimization interactively.
    """
    # Set random seed
    np.random.seed(random_state)
    
    # Generate fitness landscape
    global x, X_grid, Y_grid_true
    fitness_function = generate_fitness_landscape(dimensions, epistasis, ruggedness, sparsity)
    
    # Define the search space
    x = np.linspace(-2, 2, 50)
    if dimensions == 1:
        X_grid = x.reshape(-1, 1)
    else:
        grid_mesh = np.meshgrid(*[x]*dimensions)
        X_grid = np.vstack([g.ravel() for g in grid_mesh]).T
    
    Y_grid_true = fitness_function(X_grid)
    
    # Initialize samples
    X_sample = initialize_samples(initial_samples, dimensions, sampling_method)
    Y_sample = fitness_function(X_sample)
    
    # Select surrogate model
    model = select_surrogate_model(surrogate_model, kernel_choice, length_scale, noise_level)
    
    # Perform Bayesian optimization
    X_sample, Y_sample, mu, sigma, acquisition, idx_next = bayesian_optimization(
        fitness_function, X_sample, Y_sample, model, surrogate_model,
        acquisition_function, iterations, batch_size, kappa, xi
    )
    
    # Choose color map
    if color_scheme == 'Blue':
        color_map = cm.Blues
    elif color_scheme == 'Viridis':
        color_map = cm.viridis
    elif color_scheme == 'Plasma':
        color_map = cm.plasma
    elif color_scheme == 'Grey':
        color_map = cm.Greys
    elif color_scheme == 'Cividis':
        color_map = cm.cividis
    elif color_scheme == 'Magma':
        color_map = cm.magma
    else:
        color_map = cm.Blues  # Default to Blue
    
    # Plot the results
    plot_results(X_sample, Y_sample, mu, sigma, acquisition, dimensions, surrogate_model, acquisition_function, color_map, batch_size, idx_next)

def reset_parameters(_):
    """
    Resets the parameters to their default values.
    """
    acquisition_widget.value = 'UCB'
    surrogate_widget.value = 'Gaussian Process'
    kernel_widget.value = 'Matern'
    kappa_widget.value = 2.0
    xi_widget.value = 0.01
    epistasis_widget.value = 1.0
    ruggedness_widget.value = 1.0
    sparsity_widget.value = 0.0
    iterations_widget.value = 5
    batch_widget.value = 1
    dimensions_widget.value = 1
    initial_samples_widget.value = 5
    sampling_method_widget.value = 'Random'
    length_scale_widget.value = 1.0
    noise_level_widget.value = 1e-5
    random_state_widget.value = 42
    color_scheme_widget.value = 'Blue'

# Create interactive widgets with bold labels and explanations
style = {'description_width': 'initial'}
layout = Layout(width='60%')

def create_bold_label(text):
    return widgets.HTML(value=f"<b>{text}</b>")

acquisition_widget = widgets.Dropdown(
    options=[
        ('Upper Confidence Bound (UCB)', 'UCB'),
        ('Expected Improvement (EI)', 'Expected Improvement'),
        ('Probability of Improvement (PI)', 'Probability of Improvement'),
        ('Thompson Sampling', 'Thompson Sampling'),
        ('Random Sampling', 'Random')
    ],
    value='UCB',
    description='',
    style=style,
    layout=layout,
    tooltip='Select the acquisition function guiding the next sampling point.'
)
acquisition_label = create_bold_label('Acquisition Function:')
acquisition_expl = widgets.Label("Select the acquisition function guiding the next sampling point.")

surrogate_widget = widgets.Dropdown(
    options=['Gaussian Process', 'Random Forest', 'Neural Network'],
    value='Gaussian Process',
    description='',
    style=style,
    layout=layout,
    tooltip='Choose the model to approximate the unknown fitness function.'
)
surrogate_label = create_bold_label('Surrogate Model:')
surrogate_expl = widgets.Label("Choose the model to approximate the unknown fitness function.")

kernel_widget = widgets.Dropdown(
    options=['Matern', 'RBF', 'RationalQuadratic'],
    value='Matern',
    description='',
    style=style,
    layout=layout,
    tooltip='Select the kernel function for the Gaussian Process.'
)
kernel_label = create_bold_label('GP Kernel:')
kernel_expl = widgets.Label("Select the kernel function for the Gaussian Process.")

kappa_widget = widgets.FloatSlider(
    min=0.1, max=5.0, step=0.1, value=2.0,
    description='',
    style=style,
    layout=layout,
    tooltip='Controls exploration vs. exploitation in UCB.'
)
kappa_label = create_bold_label('Exploration (kappa):')
kappa_expl = widgets.Label("Controls exploration vs. exploitation in UCB.")

xi_widget = widgets.FloatSlider(
    min=0.0, max=1.0, step=0.01, value=0.01,
    description='',
    style=style,
    layout=layout,
    tooltip='Controls exploration in EI and PI acquisition functions.'
)
xi_label = create_bold_label('Exploration (xi):')
xi_expl = widgets.Label("Controls exploration in EI and PI acquisition functions.")

epistasis_widget = widgets.FloatSlider(
    min=0.0, max=2.0, step=0.1, value=1.0,
    description='',
    style=style,
    layout=layout,
    tooltip='Adjusts interaction between variables in the fitness landscape.'
)
epistasis_label = create_bold_label('Epistasis Level:')
epistasis_expl = widgets.Label("Adjusts interaction between variables in the fitness landscape.")

ruggedness_widget = widgets.FloatSlider(
    min=0.0, max=2.0, step=0.1, value=1.0,
    description='',
    style=style,
    layout=layout,
    tooltip='Controls complexity of the fitness landscape.'
)
ruggedness_label = create_bold_label('Ruggedness Level:')
ruggedness_expl = widgets.Label("Controls complexity of the fitness landscape.")

sparsity_widget = widgets.FloatSlider(
    min=0.0, max=1.0, step=0.05, value=0.0,
    description='',
    style=style,
    layout=layout,
    tooltip='Introduces inactive regions in the landscape.'
)
sparsity_label = create_bold_label('Sparsity Level:')
sparsity_expl = widgets.Label("Introduces inactive regions in the landscape.")

iterations_widget = widgets.IntSlider(
    min=1, max=20, step=1, value=5,
    description='',
    style=style,
    layout=layout,
    tooltip='Number of optimization steps.'
)
iterations_label = create_bold_label('Optimization Iterations:')
iterations_expl = widgets.Label("Number of optimization steps.")

batch_widget = widgets.IntSlider(
    min=1, max=5, step=1, value=1,
    description='',
    style=style,
    layout=layout,
    tooltip='Number of samples selected per iteration.'
)
batch_label = create_bold_label('Batch Size:')
batch_expl = widgets.Label("Number of samples selected per iteration.")

dimensions_widget = widgets.IntSlider(
    min=1, max=2, step=1, value=2,
    description='',
    style=style,
    layout=layout,
    tooltip='Select 1D or 2D optimization (visualization supported).'
)
dimensions_label = create_bold_label('Problem Dimensions:')
dimensions_expl = widgets.Label("Select 1D or 2D optimization (visualization supported).")

initial_samples_widget = widgets.IntSlider(
    min=1, max=20, step=1, value=5,
    description='',
    style=style,
    layout=layout,
    tooltip='Number of initial samples.'
)
initial_samples_label = create_bold_label('Initial Samples:')
initial_samples_expl = widgets.Label("Number of initial samples.")

sampling_method_widget = widgets.Dropdown(
    options=['Random', 'Latin Hypercube', 'Sobol'],
    value='Random',
    description='',
    style=style,
    layout=layout,
    tooltip='Method to generate initial samples.'
)
sampling_label = create_bold_label('Sampling Method:')
sampling_expl = widgets.Label("Method to generate initial samples.")

length_scale_widget = widgets.FloatSlider(
    min=0.1, max=10.0, step=0.1, value=1.0,
    description='',
    style=style,
    layout=layout,
    tooltip='Determines smoothness of GP predictions.'
)
length_scale_label = create_bold_label('GP Length Scale:')
length_scale_expl = widgets.Label("Determines smoothness of GP predictions.")

noise_level_widget = widgets.FloatLogSlider(
    min=-10, max=-2, step=0.1, value=1e-5,
    description='',
    style=style,
    layout=layout,
    tooltip='Assumed noise level in GP observations.'
)
noise_level_label = create_bold_label('GP Noise Level:')
noise_level_expl = widgets.Label("Assumed noise level in GP observations.")

random_state_widget = widgets.IntText(
    value=42,
    description='',
    style=style,
    layout=layout,
    tooltip='Seed for random number generators.'
)
random_state_label = create_bold_label('Random Seed:')
random_state_expl = widgets.Label("Seed for random number generators.")

color_scheme_widget = widgets.Dropdown(
    options=['Blue', 'Viridis', 'Plasma', 'Grey', 'Cividis', 'Magma'],
    value='Blue',
    description='',
    style=style,
    layout=layout,
    tooltip='Choose the color scheme for the plots.'
)
color_scheme_label = create_bold_label('Color Scheme:')
color_scheme_expl = widgets.Label("Choose the color scheme for the plots.")

# Reset Button
reset_button = Button(description="Reset Parameters", tooltip='Reset all parameters to default values.')
reset_button.on_click(reset_parameters)

# Organize widgets into tabs for better UX
tab_nest = widgets.Tab()

# Tab 1: Optimization Settings
tab1_contents = VBox([
    HBox([acquisition_label, acquisition_widget]),
    acquisition_expl,
    HBox([surrogate_label, surrogate_widget]),
    surrogate_expl,
    HBox([kernel_label, kernel_widget]),
    kernel_expl,
    HBox([kappa_label, kappa_widget]),
    kappa_expl,
    HBox([xi_label, xi_widget]),
    xi_expl,
    HBox([iterations_label, iterations_widget]),
    iterations_expl,
    HBox([batch_label, batch_widget]),
    batch_expl
])

# Tab 2: Fitness Landscape Settings
tab2_contents = VBox([
    HBox([epistasis_label, epistasis_widget]),
    epistasis_expl,
    HBox([ruggedness_label, ruggedness_widget]),
    ruggedness_expl,
    HBox([sparsity_label, sparsity_widget]),
    sparsity_expl,
    HBox([dimensions_label, dimensions_widget]),
    dimensions_expl
])

# Tab 3: Sampling Settings
tab3_contents = VBox([
    HBox([initial_samples_label, initial_samples_widget]),
    initial_samples_expl,
    HBox([sampling_label, sampling_method_widget]),
    sampling_expl,
    HBox([random_state_label, random_state_widget]),
    random_state_expl
])

# Tab 4: Advanced Settings
tab4_contents = VBox([
    HBox([length_scale_label, length_scale_widget]),
    length_scale_expl,
    HBox([noise_level_label, noise_level_widget]),
    noise_level_expl,
    HBox([color_scheme_label, color_scheme_widget]),
    color_scheme_expl
])

tab_nest.children = [tab1_contents, tab2_contents, tab3_contents, tab4_contents]
tab_nest.set_title(0, 'Optimization')
tab_nest.set_title(1, 'Fitness Landscape')
tab_nest.set_title(2, 'Sampling')
tab_nest.set_title(3, 'Advanced')

# Interactive output
out = interactive_output(
    interactive_bayesian_optimization,
    {
        'acquisition_function': acquisition_widget,
        'surrogate_model': surrogate_widget,
        'kernel_choice': kernel_widget,
        'kappa': kappa_widget,
        'xi': xi_widget,
        'epistasis': epistasis_widget,
        'ruggedness': ruggedness_widget,
        'sparsity': sparsity_widget,
        'iterations': iterations_widget,
        'batch_size': batch_widget,
        'dimensions': dimensions_widget,
        'initial_samples': initial_samples_widget,
        'sampling_method': sampling_method_widget,
        'length_scale': length_scale_widget,
        'noise_level': noise_level_widget,
        'random_state': random_state_widget,
        'color_scheme': color_scheme_widget
    }
)

# Display the UI and output
ui = VBox([tab_nest, reset_button])
display(ui, out)
